In [36]:
#Importando as bibliotecas necessárias
import pandas as pd
import psycopg2 as psy
import os
from dotenv import load_dotenv

try:

    load_dotenv() # - carregando as variáveis de ambiente para que possam ser utilizadas
    
    conexao = psy.connect(
        dbname = os.getenv("DB_NAME"),
        user = os.getenv("USER"),
        password = os.getenv("PASSWORD"),
        host = os.getenv("HOST"),
        port = os.getenv("PORT")
    ) # - estabelecendo a conexão com o banco de dados

    cursor = conexao.cursor()

    #Execução do comando para trazer os dados da tabela original no banco de dados
    cursor.execute(
        '''SELECT * FROM super_adm;'''
    )

    #Criando um dataframe apartir dos dados capturados
    df = pd.DataFrame(cursor.fetchall(), columns=[item[0] for item in cursor.description])


    cursor.close() # - fechando conexão
    conexao.close() # - fechando conexão

except Exception as e:
    print(f"Erro ao conectar com o banco: "+str(e)) # - tratando exceção, (caso ocorra)

In [37]:
#Realizando as pseudonimizaçãoes necessárias
#Criando um clone do dataframe original
df_super_pseudo = df.copy()

#pseudonimização da coluna de id - sem necessidade
#pseudonimização da coluna de nome
df_super_pseudo['nome'] = ['User'+str(cont).zfill(3) for cont in range(len(df_super_pseudo))]
#pseudonimização da coluna de cargo
df_super_pseudo['cargo'] = df_super_pseudo['cargo'].apply(lambda cargo: cargo[:3]+f'{"*" * (len(cargo)-3)}')
#pseudonimização da coluna de email
df_super_pseudo['email'] = df_super_pseudo['email'].apply(lambda email: email[:2] + f"{'*' * (email.find('@') -2)}"+email[email.find('@'):])
#pseudonimização da coluna de senha_hash - criptografia


In [38]:
#Renomeando as colunas do dataframe
df_super_pseudo.rename(columns={
    "id":"id",
    "nome":"nome_pseudo",
    "cargo" : "cargo_pseudo",
    "email": "email_pseudo",
    "senha": "senha_criptografada"
}, inplace=True)

In [39]:
#realizando o display do dataframe
display(df_super_pseudo)

,id,nome_pseudo,cargo_pseudo,email_pseudo,senha_criptografada
0,1,User000,Dev********,lu**********@germinare.org.br,$2a$15$.RNms4TylNvpnjxb3FVP0eK8NMw9zXVbchFaZW6...
1,2,User001,Dev********,ry*********@germinare.org.br,$2a$15$pSyKf8/wrB7lLO6YrYlHL.hfDiFeIiKsqtV0Gmz...
2,3,User002,Dev*********,gu*********@germinare.org.br,$2a$15$CweaGwjD9fESqkuFqFliJeyLEy1Vv6cBYx.05bL...
3,4,User003,Dev*********,lu**********@germinare.org.br,$2a$15$6pG3a8QJkb9F2jcHhtIl..WeLMoohFveGlqs8XJ...
4,5,User004,Ana**************,en**********@germinare.org.br,$2a$15$4CHCAC2RhRkDnPXvOt5JDuM5/pfBTTa09.9oTbT...
5,6,User005,UX ******************************,le*************@germinare.org.br,$2a$15$3g5smqPP7tPeNpDjyGUiKumKB9yIsELtWM9g.31...
6,7,User006,Eng****************,ni*********@germinare.org.br,$2a$15$SbjpG85.RrTeBLkMZY8HkeRzaoAJCpTUHdxvUMv...
7,8,User007,Eng****************,da********@germinare.org.br,$2a$15$hZ0zxuAZin4X0vtq54DKEOYVaSXpYnPseLoHElA...


In [40]:
#Transferindo todas as informações do dataframe pseudonimizado para o banco de dados 


try:
    conexao = psy.connect(
        dbname = os.getenv("DB_PSEUDO"),
        user = os.getenv("USER"),
        password = os.getenv("PASSWORD"),
        host = os.getenv("HOST"),
        port = os.getenv("PORT")
    ) # - estabelecendo a conexão com o banco de dados

    cursor = conexao.cursor()

    #execução do comando para trazer os dados da tabela original
    query = """
    DELETE FROM super_adm_pseudo;
    """ # - comando que vai ser aplicado

    valores = [tuple(x) for x in df_super_pseudo.to_numpy()] # - transformando cada linha do dataframe em uma tupla

    for c in valores:
        cursor.execute(query, c) # - executando o comando no banco de dados

    conexao.commit() # - salvando as alterações


    cursor.close() # - fechando conexão 
    conexao.close() # - fechando conexão 

except Exception as e:
    print(f"Erro ao conectar com o banco: "+str(e)) # - tratando exceção, (caso ocorra)

In [41]:
#Transferindo todas as informações do dataframe pseudonimizado para o banco de dados 


try:
    conexao = psy.connect(
        dbname = os.getenv("DB_PSEUDO"),
        user = os.getenv("USER"),
        password = os.getenv("PASSWORD"),
        host = os.getenv("HOST"),
        port = os.getenv("PORT")
    ) # - estabelecendo a conexão com o banco de dados

    cursor = conexao.cursor()

    #execução do comando para trazer os dados da tabela original
    query = """
    INSERT INTO super_adm_pseudo (id, nome_pseudo, cargo_pseudo, email_pseudo, senha_criptografada)
    VALUES (%s, %s, %s, %s, %s) """ # - comando que vai ser aplicado

    valores = [tuple(x) for x in df_super_pseudo.to_numpy()] # - transformando cada linha do dataframe em uma tupla

    for c in valores:
        cursor.execute(query, c) # - executando o comando no banco de dados

    conexao.commit() # - salvando as alterações


    cursor.close() # - fechando conexão 
    conexao.close() # - fechando conexão 

except Exception as e:
    print(f"Erro ao conectar com o banco: "+str(e)) # - tratando exceção, (caso ocorra)

In [42]:
#Tranferindo as informações originais e pseudonimizadas para arquivos csv
df.to_csv('CSV/originais_superadm.csv')
df_super_pseudo.to_csv('CSV/pseudo_superadm.csv')